<a href="https://colab.research.google.com/github/MDankloff/Cluster-Bias-Disco/blob/main/BAF_Reproduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

import glob
import os

# Load Bank Account Fraud (BAF) Tabular Dataset Suite

In [ ]:
dir_path = '/content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF'

'''if os.path.exists(dir_path):
    print("Directory exists")
    print("Files in directory:")
    print(os.listdir(dir_path))  # This will list all files in the directory
else:
    print("Directory does not exist")'''

#The parquet extensions allow for smaller file sizes (approx. 3 times smaller) and quicker load speeds

extension = "csv" #"parquet" depending on the downloaded file
data_paths = glob.glob(f"{dir_path}/*.{extension}")

'''# Check if any CSV files are found
if not data_paths:
    raise FileNotFoundError(f"No files with extension '{extension}' found in the directory.")
else:
    print("CSV files found:")
    print(data_paths)'''

def read_dataset(path, ext = extension):
    #return pd.read_csv(path)
    if ext == "csv":
      return pd.read_csv(path)
    elif ext == "parquet":
      return pd.read_excel(path)
    else:
      raise ValueError(f"Unsupported file extension: {ext}")

# Extract variant name from the file path (without the extension)
def get_variant(path):
    return os.path.basename(path).split(".")[0]

# Dictionary comprehension to read all CSV files into a dictionary of DataFrames
dataframes = {
    get_variant(path): read_dataset(path) for path in data_paths
}
print(f"Loaded datasets: {list(dataframes.keys())}")

Loaded datasets: ['Base', 'Variant I', 'Variant II', 'Variant III', 'Variant IV', 'Variant V']


In [ ]:
base = dataframes['Base'] # sampled to best represent original dataset
variant1 = dataframes['Variant I'] # higher group size disparity than base - reducing the size of the minority group from approx 20 - 10% of the dataset
variant2 = dataframes['Variant II'] # higher prevalance disparity than base - one group has 5 x the fraud detection rate of the other while group sizes are equal
variant3 = dataframes['Variant III'] # better separability of one of the groups -
variant4 = dataframes['Variant IV'] # higher prevalance disparity in train
variant5 = dataframes['Variant V'] # better separability in train for one of the groups
#base.info()
variant1.info()
#base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000